In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
import os

project_path = '/content/drive/Othercomputers/My laptop/DestopBackup/יד/Anomaly_Detection_System_Project/Detection-Engine'

if os.path.exists(project_path):
    os.chdir(project_path)
    sys.path.append(project_path)
    print(f"Successfully entered the folder: {os.getcwd()}")
    # בדיקה שרואים את הקבצים
    print(f"Files in folder: {os.listdir('.')}")
else:
    print("Path not found.")

Successfully entered the folder: /content/drive/Othercomputers/My laptop/DestopBackup/יד/Anomaly_Detection_System_Project/Detection-Engine
Files in folder: ['Datasets', 'scripts', 'venv', 'models', 'data', 'utils', 'requirements.txt', 'train_packet_model_2.py', 'train_flow_model_1.py', 'train_policy_model.py', 'train_colab']


In [5]:
#----- Model 1 -----

import time
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import numpy as np
import os
from models.model import RBM
from utils.early_stopping import EarlyStopping

def train_flow_rbm_colab():
    # Hyperparameters
    BATCH_SIZE = 512
    HIDDEN_UNITS = 128
    LEARNING_RATE = 0.0002
    EPOCHS = 300
    CD_K = 20
    VAL_SPLIT = 0.1

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"--- Training Model 1 (Flow) on {device} ---")

    # Data Loading
    data = np.load('data/processed/network/flow_train.npy')
    data_tensor = torch.from_numpy(data.astype(np.float32)).to(device)
    input_mean_val = data_tensor.mean().item()

    train_size = int((1-VAL_SPLIT) * len(data_tensor))
    train_subset, val_subset = random_split(data_tensor, [train_size, len(data_tensor) - train_size])
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)

    rbm = RBM(data.shape[1], HIDDEN_UNITS, k=CD_K).to(device)
    optimizer = torch.optim.Adam(rbm.parameters(), lr=LEARNING_RATE)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)


    timestamp = time.strftime("%Y%m%d-%H%M%S")
    model_path = f'models/artifacts/flow_model_{timestamp}.pth'
    stopper = EarlyStopping(patience=25, path=model_path)

    epoch_times = []

    for epoch in range(EPOCHS):
        start_time = time.time()

        # Training Phase
        rbm.train()
        train_mse = 0
        for batch in train_loader:
            v_pos, v_neg = rbm(batch)
            loss = rbm.free_energy(v_pos) - rbm.free_energy(v_neg)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_mse += F.mse_loss(v_pos, v_neg).item()

        # Validation Phase
        rbm.eval()
        val_mses = []
        with torch.no_grad():
            for batch in val_loader:
                v_pos, v_neg = rbm(batch)
                val_mses.append(F.mse_loss(v_pos, v_neg).item())

        duration = time.time() - start_time
        epoch_times.append(duration)
        avg_val_mse = sum(val_mses) / len(val_mses)

        # Update Scheduler
        scheduler.step(avg_val_mse)
        current_lr = optimizer.param_groups[0]['lr']

        # Calculate Reconstruction Error %
        recon_error_pct = (avg_val_mse / input_mean_val) * 100 if input_mean_val != 0 else 0

        print(f"Epoch [{epoch+1}/{EPOCHS}] | Val MSE: {avg_val_mse:.6f} | Error: {recon_error_pct:.2f}% | LR: {current_lr:.6f} | Time: {duration:.2f}s")

        stopper(avg_val_mse, rbm)
        if stopper.early_stop:
            print(f"--> Convergence reached at epoch {epoch+1}.")
            break

    # 3. Comprehensive Final Metrics
    print_final_metrics("Flow Model (1)", epoch_times, val_mses)

def print_final_metrics(name, times, mses):
    mses = np.array(mses)
    # The "3-Sigma Rule": Covering 99.7% of the normal distribution
    mean_mse = mses.mean()
    std_mse = mses.std()
    threshold = mean_mse + (3 * std_mse)

    print("\n" + "="*45)
    print(f"      {name} FINAL ANALYSIS")
    print("="*45)
    print(f"Total Training Time: {sum(times):.2f}s")
    print(f"Average Epoch Duration: {sum(times)/len(times):.2f}s")
    print(f"Final Validation MSE (μ): {mean_mse:.6f}")
    print(f"MSE Standard Deviation (σ): {std_mse:.6f}")
    print(f"Calculated Anomaly Threshold (μ + 3σ): {threshold:.6f}")
    print("-" * 45)
    print("STATISTICAL LOGIC:")
    print("1. MSE Standard Deviation measures model stability.")
    print("2. The threshold is your 'Mathematical Shield' - any")
    print("   value above it is flagged as a likely attack.")
    print("="*45)

if __name__ == "__main__":
    train_flow_rbm_colab()

--- Training Model 1 (Flow) on cuda ---
Epoch [1/300] | Val MSE: 0.054480 | Error: 98.29% | LR: 0.000200 | Time: 12.35s
-> Validation energy improved. Saving model artifact...
Epoch [2/300] | Val MSE: 0.047254 | Error: 85.25% | LR: 0.000200 | Time: 12.00s
-> Validation energy improved. Saving model artifact...
Epoch [3/300] | Val MSE: 0.046095 | Error: 83.16% | LR: 0.000200 | Time: 12.18s
-> Validation energy improved. Saving model artifact...
Epoch [4/300] | Val MSE: 0.045767 | Error: 82.57% | LR: 0.000200 | Time: 11.92s
-> Validation energy improved. Saving model artifact...
Epoch [5/300] | Val MSE: 0.045056 | Error: 81.29% | LR: 0.000200 | Time: 12.07s
-> Validation energy improved. Saving model artifact...
Epoch [6/300] | Val MSE: 0.045658 | Error: 82.37% | LR: 0.000200 | Time: 11.87s
Early Stopping counter: 1 out of 25
Epoch [7/300] | Val MSE: 0.044875 | Error: 80.96% | LR: 0.000200 | Time: 11.85s
-> Validation energy improved. Saving model artifact...
Epoch [8/300] | Val MSE: 0.0

In [ ]:
#----- Model 2 -----

import time
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import numpy as np
import os
from models.model import RBM
from utils.early_stopping import EarlyStopping

def train_packet_model_colab():
    BATCH_SIZE = 512
    VISIBLE_UNITS = 12
    HIDDEN_UNITS = 256
    LEARNING_RATE = 0.0002
    EPOCHS = 500
    CD_K = 20
    VAL_SPLIT = 0.1

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"--- Training Model 2 (Packet) on {device} ---")

    data_path = 'data/processed/network/packet_train.npy'

    if not os.path.exists(data_path):
      print(f"Error: {data_path} not found. Please check your data directory.")

    data = np.load(data_path)
    data_tensor = torch.from_numpy(data.astype(np.float32)).to(device)
    input_mean_val = data_tensor.mean().item()

    train_size = int(0.9 * len(data_tensor))
    train_subset, val_subset = random_split(data_tensor, [train_size, len(data_tensor) - train_size])
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)

    rbm = RBM(VISIBLE_UNITS, HIDDEN_UNITS, k=CD_K).to(device)
    optimizer = torch.optim.Adam(rbm.parameters(), lr=LEARNING_RATE)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)

    timestamp = time.strftime("%Y%m%d-%H%M%S")
    model_path = f'models/artifacts/packet_model_{timestamp}.pth'
    stopper = EarlyStopping(patience=250, path=model_path)

    epoch_times = []
    print(f"Saving model to {model_path}")

    for epoch in range(EPOCHS):
        start_time = time.time()

        # Training Phase
        rbm.train()
        train_mse = 0
        for batch in train_loader:
            v_pos, v_neg = rbm(batch)
            loss = rbm.free_energy(v_pos) - rbm.free_energy(v_neg)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_mse += F.mse_loss(v_pos, v_neg).item()

        # Validation Phase
        rbm.eval()
        val_mses = []
        with torch.no_grad():
            for batch in val_loader:
                v_pos, v_neg = rbm(batch)
                val_mses.append(F.mse_loss(v_pos, v_neg).item())

        duration = time.time() - start_time
        epoch_times.append(duration)
        avg_val_mse = sum(val_mses) / len(val_mses)

        # Update Scheduler
        scheduler.step(avg_val_mse)
        current_lr = optimizer.param_groups[0]['lr']

        # Calculate Reconstruction Error % relative to input mean
        recon_error_pct = (avg_val_mse / input_mean_val) * 100 if input_mean_val != 0 else 0

        print(f"Epoch [{epoch+1}/{EPOCHS}] | Val MSE: {avg_val_mse:.6f} | Error: {recon_error_pct:.2f}% | LR: {current_lr:.6f} | Time: {duration:.2f}s")

        stopper(avg_val_mse, rbm)
        if stopper.early_stop:
            print(f"--> Optimized Convergence reached at epoch {epoch+1}.")
            break

    # 3. Final Statistics Summary
    print_packet_stats(epoch_times, val_mses)

def print_packet_stats(times, mses):
    """
    Prints a detailed analytical summary of the trained Packet model.
    Includes time performance and statistical anomaly thresholds.
    """
    mses = np.array(mses)
    mean_mse = mses.mean()
    std_mse = mses.std()

    # Statistical Anomaly Threshold: μ + 3σ
    # Theoretically covers 99.7% of the normal distribution
    threshold = mean_mse + (3 * std_mse)

    print("\n" + "="*45)
    print("      PACKET MODEL (2) FINAL ANALYSIS")
    print("="*45)
    print(f"Total Training Time: {sum(times):.2f}s")
    print(f"Average Epoch Duration: {sum(times)/len(times):.2f}s")
    print(f"Final Validation MSE (μ): {mean_mse:.6f}")
    print(f"MSE Standard Deviation (σ): {std_mse:.6f}")
    print(f"Calculated Anomaly Threshold (μ + 3σ): {threshold:.6f}")
    print("-" * 45)
    print("TECHNICAL SUMMARY (English):")
    print("1. Standard Deviation (σ): Low values indicate high consistency.")
    print("2. Threshold (μ + 3σ): Acts as the decision boundary.")
    print("3. Status: Ready for Inference Engine integration.")
    print("="*45)

if __name__ == "__main__":
    train_packet_model_colab()

--- Training Model 2 (Packet) on cuda ---
Epoch [1/300] | Val MSE: 0.110937 | Error: 46.99% | Time: 8.92s
-> Validation energy improved. Saving model artifact...
Epoch [2/300] | Val MSE: 0.110690 | Error: 46.88% | Time: 7.96s
-> Validation energy improved. Saving model artifact...
Epoch [3/300] | Val MSE: 0.109940 | Error: 46.56% | Time: 6.98s
-> Validation energy improved. Saving model artifact...
Epoch [4/300] | Val MSE: 0.109717 | Error: 46.47% | Time: 8.11s
-> Validation energy improved. Saving model artifact...
Epoch [5/300] | Val MSE: 0.109555 | Error: 46.40% | Time: 6.98s
-> Validation energy improved. Saving model artifact...
Epoch [6/300] | Val MSE: 0.110757 | Error: 46.91% | Time: 8.01s
Early Stopping counter: 1 out of 20
Epoch [7/300] | Val MSE: 0.109427 | Error: 46.35% | Time: 7.63s
-> Validation energy improved. Saving model artifact...
Epoch [8/300] | Val MSE: 0.108880 | Error: 46.11% | Time: 7.43s
-> Validation energy improved. Saving model artifact...
Epoch [9/300] | Va

In [4]:
#----- Model 3 -----

import torch
import torch.nn.functional as F
import time
from torch.utils.data import DataLoader, random_split
import numpy as np
from models.model import RBM
from utils.early_stopping import EarlyStopping

def train_policy_colab():
    # היפר-פרמטרים משופרים (יותר חזקים מהריצה המקומית)
    BATCH_SIZE = 256
    HIDDEN_UNITS = 128     # הגדלנו כדי לתפוס דפוסי התנהגות מורכבים ב-CERT
    LEARNING_RATE = 0.001 # קצב למידה יציב
    EPOCHS = 300           # יש לנו כוח חישוב, בוא ננצל אותו לשיפור מקסימלי
    CD_K = 20              # דגימה עמוקה לזיהוי אנומליות מדויק
    VAL_SPLIT = 0.1

    # הגדרת ה-GPU ב-Colab
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"--- Training Model 3 (Policy) on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'} ---")

    # טעינת הנתונים מהנתיב היחסי (בזכות ה-os.chdir)
    data_path = 'data/processed/policy/policy_train.npy'
    data = np.load(data_path)
    data_tensor = torch.from_numpy(data.astype(np.float32)).to(device)
    input_mean_val = data_tensor.mean().item()

    train_size = int((1 - VAL_SPLIT) * len(data_tensor))
    train_subset, val_subset = random_split(data_tensor, [train_size, len(data_tensor) - train_size])
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)

    visible_units = data.shape[1]
    rbm = RBM(visible_units, HIDDEN_UNITS, k=CD_K).to(device)
    optimizer = torch.optim.Adam(rbm.parameters(), lr=LEARNING_RATE)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)

    #Save the model to drive
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    model_path = f'models/artifacts/policy_model_{timestamp}.pth'
    stopper = EarlyStopping(patience=25, path=model_path)

    epoch_times = []

    for epoch in range(EPOCHS):
        start_time = time.time()
        rbm.train()
        train_mse = 0
        for batch in train_loader:
            v_pos, v_neg = rbm(batch)
            loss = rbm.free_energy(v_pos) - rbm.free_energy(v_neg)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_mse += F.mse_loss(v_pos, v_neg).item()

        rbm.eval()
        val_mses = []
        with torch.no_grad():
            for batch in val_loader:
                v_pos, v_neg = rbm(batch)
                val_mses.append(F.mse_loss(v_pos, v_neg).item())

        duration = time.time() - start_time
        epoch_times.append(duration)
        avg_val = sum(val_mses) / len(val_mses)

        # Step the scheduler
        scheduler.step(avg_val)
        current_lr = optimizer.param_groups[0]['lr']

        # Calculate Reconstruction Error %
        recon_error_pct = (avg_val / input_mean_val) * 100 if input_mean_val != 0 else 0

        print(f"Epoch [{epoch+1}/{EPOCHS}] | Val MSE: {avg_val:.6f} | Error: {recon_error_pct:.2f}% | LR: {current_lr:.6f} | Time: {duration:.2f}s")

        stopper(avg_val, rbm)
        if stopper.early_stop:
            print(f"--> Convergence reached. Model 3 saved.")
            break

    # Final Statistics Block
    print_final_summary("Policy Model (3)", epoch_times, val_mses)

def print_final_summary(name, times, mses):
    mses = np.array(mses)
    # Statistical Anomaly Threshold: Mean + 3*Standard Deviation
    mean_mse = mses.mean()
    std_mse = mses.std()
    threshold = mean_mse + (3 * std_mse)

    print("\n" + "="*40)
    print(f"      {name} FINAL ANALYSIS")
    print("="*40)
    print(f"Average Epoch Duration: {sum(times)/len(times):.2f}s")
    print(f"Final Validation MSE (μ): {mean_mse:.6f}")
    print(f"MSE Standard Deviation (σ): {std_mse:.6f}")
    print(f"Calculated Anomaly Threshold (μ + 3σ): {threshold:.6f}")
    print("-" * 40)
    print("LOGIC EXPLANATION (English):")
    print("1. Standard Deviation (σ): Measures model consistency. Lower is better.")
    print("2. Threshold (μ + 3σ): Statistically covers 99.7% of normal behavior.")
    print("3. Any MSE > Threshold in real-time is flagged as an anomaly.")
    print("="*40)

if __name__ == "__main__":
    train_policy_colab()

--- Training Model 3 (Policy) on: Tesla T4 ---
Epoch [1/300] | Val MSE: 0.186092 | Error: 79.41% | LR: 0.001000 | Time: 24.25s
-> Validation energy improved. Saving model artifact...
Epoch [2/300] | Val MSE: 0.174571 | Error: 74.50% | LR: 0.001000 | Time: 20.03s
-> Validation energy improved. Saving model artifact...
Epoch [3/300] | Val MSE: 0.180979 | Error: 77.23% | LR: 0.001000 | Time: 21.00s
Early Stopping counter: 1 out of 25
Epoch [4/300] | Val MSE: 0.148543 | Error: 63.39% | LR: 0.001000 | Time: 20.54s
-> Validation energy improved. Saving model artifact...
Epoch [5/300] | Val MSE: 0.116724 | Error: 49.81% | LR: 0.001000 | Time: 20.65s
-> Validation energy improved. Saving model artifact...
Epoch [6/300] | Val MSE: 0.099719 | Error: 42.55% | LR: 0.001000 | Time: 21.13s
-> Validation energy improved. Saving model artifact...
Epoch [7/300] | Val MSE: 0.093252 | Error: 39.79% | LR: 0.001000 | Time: 20.18s
-> Validation energy improved. Saving model artifact...
Epoch [8/300] | Val M